In [1]:
from math import sqrt, sin, cos, asin, acos, tan, atan2, pi
import numpy as np

In [2]:
def qn_product(q1, q2):
    q1 = np.array(q1)
    q2 = np.array(q2)
    
    q1 = q1 / np.sqrt((q1**2).sum())
    q2 = q2 / np.sqrt((q2**2).sum())
    
    qw1, qx1, qy1, qz1 = q1
    qw2, qx2, qy2, qz2 = q2
    
    qw = qw1*qw2 - qx1*qx2 - qy1*qy2 - qz1*qz2
    qx = qw1*qx2 + qx1*qw2 + qy1*qz2 - qz1*qy2
    qy = qw1*qy2 - qx1*qz2 + qy1*qw2 + qz1*qx2
    qz = qw1*qz2 + qx1*qy2 - qy1*qx2 + qz1*qw2
    
    q = np.array([qw, qx, qy, qz])
    q = q / np.sqrt((q**2).sum())
    
    return q

In [3]:
def inverse_qn(q):
    qw, qx, qy, qz = q
    
    return (qw, -qx, -qy, -qz)

In [4]:
def print_rotate_angle(q0, q1):
    qr = qn_product(inverse_qn(q0), q1)
    qw, qx, qy, qz = qr
    rot_angle = acos(qw) * 2
    s_r = sin(rot_angle/2)
    t_x = acos(qx / s_r)
    t_y = acos(qy / s_r)
    t_z = acos(qz / s_r)
    print(f"rotation angle: {rot_angle*180/pi:.2f}°")
    print(f"angle between x axis and rotation axis: {t_x*180/pi:.2f}°")
    print(f"angle between y axis and rotation axis: {t_y*180/pi:.2f}°")
    print(f"angle between z axis and rotation axis: {t_z*180/pi:.2f}°")

In [5]:
def calculate_elevation(q0, q1, angle0, angle1):
    qr = qn_product(inverse_qn(q0), q1)
    qw, qx, qy, qz = qr
    
    azi0 = 3*pi/2 - angle0 * pi/180
    azi1 = 3*pi/2 - angle1 * pi/180
    
    A = 2*(qx*qy - qw*qz)
    B = 2*(qy*qz + qw*qx)
    C = (cos(azi1) - (qw**2 - qx**2 + qy**2 - qz**2)*cos(azi0))/abs(sin(azi0))
    
    if A**2 + B**2 - C**2 < 0:
        # print("invalid value!")
        return None

    cos_phi1 = (A*C - sqrt((A**2 + B**2 - C**2)*(B**2)))/(A**2 + B**2)
    cos_phi2 = (A*C + sqrt((A**2 + B**2 - C**2)*(B**2)))/(A**2 + B**2)
    # phi1 = acos(cos_phi1)
    elev1 = 90 - acos(abs(sin(azi0))*cos_phi1)*180/pi
    # phi2 = acos(cos_phi2)
    elev2 = 90 - acos(abs(sin(azi0))*cos_phi2)*180/pi

    return elev1, elev2

In [6]:
def get_final_elevation(q0, q1, q2, angle0, angle1, angle2):
    elev1 = calculate_elevation(q0, q1, angle0, angle1)
    elev2 = calculate_elevation(q0, q2, angle0, angle2)
    
    if elev1 is None and elev2 is None:
        print("cannot measure elevation!")
        return
    if elev1 is not None and elev2 is None:
        print(f"estimated elevation1: {elev1[0]:.2f}° or {elev1[1]:.2f}°")
        print("estimated elevation2 cannot be calculated")
        return
    if elev2 is not None and elev1 is None:
        print(f"estimated elevation2: {elev2[0]:.2f}° or {elev2[1]:.2f}°")
        print("estimated elevation1 is cannot be calculated")
        return
    
    print(f"estimated elevation1: {elev1[0]:.2f}° or {elev1[1]:.2f}°")
    print(f"estimated elevation2: {elev2[0]:.2f}° or {elev2[1]:.2f}°")
    
    elev1 = np.expand_dims(np.array(elev1), 0)
    elev2 = np.expand_dims(np.array(elev2), 1)
    
    diff = abs(elev1 - elev2)
    print(f"final estimated elevation: {(elev1 + elev2).flatten()[np.argmin(diff)]/2:.2f}°")

## 45

In [7]:
q0 = (0.066751931, -0.690721692, -0.043254738, -0.718731343)
q1 = (0.322023556, -0.688363174, 0.105393746, -0.641363694)
q2 = (0.139139634, 0.655033811, 0.228849868, 0.706538527)

angle10_true = -45
angle11_true = -39.7
angle12_true = -42.2
angle10 = -41.7705
angle11 = -26.3657
angle12 = -5.0957

angle20_true = 45
angle21_true = 32.6
angle22_true = 31.2
angle20 = 53.0708
angle21 = -26.3657
angle22 = -5.0957

phone_height = 93.5
distance1 = 90
height_low1 = 93.5 - phone_height
height_high1 = 104.5 - phone_height

print("distance1:", distance1)
print("height_low1:", height_low1)
print("hieght_high1:", height_high1)
print(f"elevation1: {atan2(height_low1, distance1)*180/pi:.2f}° ~ {atan2(height_high1, distance1)*180/pi:.2f}°")

print("="*50)
print("<<rotate1>>")
print_rotate_angle(q0, q1)

print("="*50)
print("<<rotate2>>")
print_rotate_angle(q0, q2)

distance1: 90
height_low1: 0.0
hieght_high1: 11.0
elevation1: 0.00° ~ 6.97°
<<rotate1>>
rotation angle: 35.13°
angle between x axis and rotation axis: 76.00°
angle between y axis and rotation axis: 95.85°
angle between z axis and rotation axis: 15.22°
<<rotate2>>
rotation angle: 327.84°
angle between x axis and rotation axis: 88.78°
angle between y axis and rotation axis: 89.16°
angle between z axis and rotation axis: 1.48°


In [8]:
print("<<angle1>>")
get_final_elevation(q0, q1, q2, angle10, angle11, angle12)

print("="*50)
print("<<angle1 true>>")
get_final_elevation(q0, q1, q2, angle10_true, angle11_true, angle12_true)

print("="*50)
print("<<angle2>>")
get_final_elevation(q0, q1, q2, angle20, angle21, angle22)

print("="*50)
print("<<angle2 true>>")
get_final_elevation(q0, q1, q2, angle20_true, angle21_true, angle22_true)

<<angle1>>
estimated elevation1: 1.19° or 19.11°
estimated elevation2 cannot be calculated
<<angle1 true>>
estimated elevation1: -14.42° or 2.70°
estimated elevation2: 7.21° or 8.58°
final estimated elevation: 4.96°
<<angle2>>
cannot measure elevation!
<<angle2 true>>
estimated elevation1: -12.63° or 4.54°
estimated elevation2: 8.02° or 9.39°
final estimated elevation: 6.28°


## high low

In [9]:
q0 = (0.117797887, -0.695981868, -0.103176333, -0.700775984)
q1 = (0.011455393, 0.666948576, 0.221069604, 0.71145665)
q2 = (0.265404316, -0.69151925, -0.050342913, -0.669943537)

angle10_true = -46.9
angle11_true = -46.5
angle12_true = -35.7
angle10 = -41.7705
angle11 = -49.0227
angle12 = -29.2427

angle20_true = 41.4
angle21_true = 50.1
angle22_true = 41.7
angle20 = 49.0227
angle21 = 45.2804
angle22 = 49.0227

phone_height = 90

distance1 = 90
height_low1 = 119.5 - phone_height
height_high1 = 130.5 - phone_height

distance2 = 90
height_low2 = 85.5 - phone_height
height_high2 = 96.5 - phone_height

print("distance1:", distance1)
print("height_low1:", height_low1)
print("hieght_high1:", height_high1)
print(f"elevation1: {atan2(height_low1, distance1)*180/pi:.2f}° ~ {atan2(height_high1, distance1)*180/pi:.2f}°")

print("="*50)
print("distance2:", distance2)
print("height_low2:", height_low2)
print("hieght_high2:", height_high2)
print(f"elevation2: {atan2(height_low2, distance2)*180/pi:.2f}° ~ {atan2(height_high2, distance2)*180/pi:.2f}°")

print("="*50)
print("<<rotate1>>")
print_rotate_angle(q0, q1)

print("="*50)
print("<<rotate2>>")
print_rotate_angle(q0, q2)

distance1: 90
height_low1: 29.5
hieght_high1: 40.5
elevation1: 18.15° ~ 24.23°
distance2: 90
height_low2: -4.5
hieght_high2: 6.5
elevation2: -2.86° ~ 4.13°
<<rotate1>>
rotation angle: 339.62°
angle between x axis and rotation axis: 88.37°
angle between y axis and rotation axis: 90.18°
angle between z axis and rotation axis: 1.64°
<<rotate2>>
rotation angle: 18.34°
angle between x axis and rotation axis: 64.17°
angle between y axis and rotation axis: 88.88°
angle between z axis and rotation axis: 25.86°


In [10]:
print("<<angle1>>")
get_final_elevation(q0, q1, q2, angle10, angle11, angle12)

print("="*50)
print("<<angle1 true>>")
get_final_elevation(q0, q1, q2, angle10_true, angle11_true, angle12_true)

print("="*50)
print("<<angle2>>")
get_final_elevation(q0, q1, q2, angle20, angle21, angle22)

print("="*50)
print("<<angle2 true>>")
get_final_elevation(q0, q1, q2, angle20_true, angle21_true, angle22_true)

<<angle1>>
estimated elevation1: 20.85° or 23.16°
estimated elevation2: 8.77° or 42.02°
final estimated elevation: 14.81°
<<angle1 true>>
estimated elevation1: 5.62° or 7.87°
estimated elevation2: 3.17° or 34.81°
final estimated elevation: 4.40°
<<angle2>>
estimated elevation1: -1.56° or 0.62°
estimated elevation2: -9.97° or 23.08°
final estimated elevation: -5.76°
<<angle2 true>>
estimated elevation1: -26.14° or -23.87°
estimated elevation2: -12.57° or 25.59°
final estimated elevation: -18.22°
